In [2]:
import tensorflow as tf
from tensorflow.keras import layers

print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


# 构建简单的模型
## 序列模型
在 Keras 中，您可以通过组合层来构建模型。模型（通常）是由层构成的图。最常见的模型类型是层的堆叠：`tf.keras.Sequential` 模型。

要构建一个简单的全连接网络（即多层感知器），请运行以下代码：

In [3]:
model = tf.keras.Sequential()
# Adds a densely-connected alyer with 64 units to the model:
model.add(layers.Dense(64,activation='relu'))
# Add another:
model.add(layers.Dense(64,activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(10,activation='softmax'))

# 配置层
我们可以使用很多 tf.keras.layers，它们具有一些相同的构造函数参数：

* `activation`：设置层的激活函数。此参数由内置函数的名称指定，或指定为可调用对象。默认情况下，系统不会应用任何激活函数。
* `kernel_initializer` 和 `bias_initializer`：创建层权重（核和偏差）的初始化方案。此参数是一个名称或可调用对象，默认为 "Glorot uniform" 初始化器。
* `kernel_regularizer` 和 `bias_regularizer`：应用层权重（核和偏差）的正则化方案，例如 L1 或 L2 正则化。默认情况下，系统不会应用正则化函数。

以下代码使用构造函数参数实例化 `tf.keras.layers. Dense` 层：

In [4]:
# Create a sigmoid layer:
layers.Dense(64,activation='sigmoid')
# Or:
layers.Dense(64,activation=tf.sigmoid)

# A linear layer with L1 regularization of factor 0.01 applied to the kernelmatrix:
layers.Dense(64,kernel_regularizer=tf.keras.regularizers.l1(0.01))

# A linear layer with L2 regulariztion of factor 0.01 applied to the bias vector:
layers.Dense(64,bias_regularizer=tf.keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
layers.Dense(64,kernel_initializer='orthogonal')

# A linear layer with a bias vector initialized to 2.0s:
layers.Dense(64,bias_initializer=tf.keras.initializers.constant(2.0))

# 训练和评估
## 设置训练流程
构建好模型后，通过调用 `compile` 方法配置该模型的学习流程：

In [6]:
model = tf.keras.Sequential([
    # Adds  a densely-connected alyer with 64 units to the model:
    layers.Dense(64,activation='relu'),
    # Add another:
    layers.Dense(64,activation='relu'),
    # Add a softmax layer with 10 output units:
    layers.Dense(10,activation='softmax')
])

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

`tf.keras.Model.compile` 采用三个重要参数：

* `optimizer`：此对象会指定训练过程。从 `tf.train` 模块向其传递优化器实例，例如 `tf.train.AdamOptimizer`、`tf.train.RMSPropOptimizer` 或 `tf.train.GradientDescentOptimizer`。
* `loss`：要在优化期间最小化的函数。常见选择包括均方误差 (mse)、`categorical_crossentropy` 和 `binary_crossentropy`。损失函数由名称或通过从 `tf.keras.losses` 模块传递可调用对象来指定。
* `metrics`：用于监控训练。它们是 `tf.keras.metrics` 模块中的字符串名称或可调用对象。

以下代码展示了配置模型以进行训练的几个示例：

In [7]:
# Configure a model for mean-squared error regression.
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
             loss='mse', # mean squared error
             metrics=['mae']) # mean absolute error

# Configure a model for categorical classification
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=[tf.keras.metrics.categorical_accuracy])

# 输入 NumPy 数据
对于小型数据集，请使用内存中的 **NumPy**数组训练和评估模型。使用 `fit` 方法使模型与训练数据“拟合”：

In [8]:
import numpy as np

data = np.random.random((1000,32))
labels = np.random.random((1000,10))

model.fit(data,labels,epochs=10,batch_size=32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1000/1000 [==============================] - 0s 200us/sample - loss: 11.5988 - categorical_accuracy: 0.0980
Epoch 2/10
1000/1000 [==============================] - 0s 35us/sample - loss: 11.5448 - categorical_accuracy: 0.1080
Epoch 3/10
1000/1000 [==============================] - 0s 40us/sample - loss: 11.5323 - categorical_accuracy: 0.1150
Epoch 4/10
1000/1000 [==============================] - 0s 44us/sample - loss: 11.5203 - categorical_accuracy: 0.1060
Epoch 5/10
1000/1000 [==============================] - 0s 35us/sample - loss: 11.5198 - categorical_accuracy: 0.1090
Epoch 6/10
1000/1000 [==============================] - 0s 43us/sample - loss: 11.5148 - categorical_accuracy: 0.1180
Epoch 7/10
1000/1000 [==============================] - 0s 35us/sample - loss: 11.5130 - categorical_accuracy: 0.1020
Epoch 8/10
1000/1000 [==============================] - 0s 41us/sample - loss: 11.5127 - categorical_accuracy: 0.1150
Epoch 9

`tf.keras.Model.fit` 采用三个重要参数：

- `epochs`：以周期为单位进行训练。一个周期是对整个输入数据的一次迭代（以较小的批次完成迭代）。
- `batch_size`：当传递 NumPy 数据时，模型将数据分成较小的批次，并在训练期间迭代这些批次。此整数指定每个批次的大小。请注意，如果样本总数不能被批次大小整除，则最后一个批次可能更小。
- `validation_data`：在对模型进行原型设计时，您需要轻松监控该模型在某些验证数据上达到的效果。传递此参数（输入和标签元组）可以让该模型在每个周期结束时以推理模式显示所传递数据的损失和指标。
下面是使用 `validation_data` 的示例：

In [9]:
import numpy as np 

data = np.random.random((1000,32))
labels = np.random.random((1000,10))

val_data = np.random.random((100,32))
val_labels = np.random.random((100,10))

model.fit(data,labels,epochs=10,batch_size=32,
         validation_data=(val_data,val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 119us/sample - loss: 11.6305 - categorical_accuracy: 0.1110 - val_loss: 11.7139 - val_categorical_accuracy: 0.1000
Epoch 2/10
1000/1000 [==============================] - 0s 42us/sample - loss: 11.6247 - categorical_accuracy: 0.1010 - val_loss: 11.7162 - val_categorical_accuracy: 0.1100
Epoch 3/10
1000/1000 [==============================] - 0s 47us/sample - loss: 11.6233 - categorical_accuracy: 0.1040 - val_loss: 11.7054 - val_categorical_accuracy: 0.0900
Epoch 4/10
1000/1000 [==============================] - 0s 42us/sample - loss: 11.6218 - categorical_accuracy: 0.1260 - val_loss: 11.7140 - val_categorical_accuracy: 0.1400
Epoch 5/10
1000/1000 [==============================] - 0s 38us/sample - loss: 11.6226 - categorical_accuracy: 0.1210 - val_loss: 11.7124 - val_categorical_accuracy: 0.1200
Epoch 6/10
1000/1000 [==============================] - 0s 43us/sample - loss: 11.6209 

### 输入 tf.data 数据集
使用 Datasets API 可扩展为大型数据集或多设备训练。将 `tf.data.Dataset` 实例传递到 `fit` 方法：

In [10]:
# Instantiates a toy dataset instance:
dataset = tf.data.Dataset.from_tensor_slices((data,labels))
dataset = dataset.batch(32)
dataset = dataset.repeat()

#Don't forget to specify `steps_per_epoch` when calling `fit` on a dataset.
model.fit(dataset,epochs=10,steps_per_epoch=30)

Epoch 1/10
30/30 [==============================] - 0s 7ms/step - loss: 11.5950 - categorical_accuracy: 0.1479
Epoch 2/10
30/30 [==============================] - 0s 1ms/step - loss: 11.6286 - categorical_accuracy: 0.1453
Epoch 3/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5978 - categorical_accuracy: 0.1656
Epoch 4/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5715 - categorical_accuracy: 0.1656
Epoch 5/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5883 - categorical_accuracy: 0.1496
Epoch 6/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5664 - categorical_accuracy: 0.1453
Epoch 7/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5362 - categorical_accuracy: 0.1592
Epoch 8/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5582 - categorical_accuracy: 0.1731
Epoch 9/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5543 - categorical_accuracy: 0.1656
E

在上方代码中，fit 方法使用了 `steps_per_epoch` 参数（表示模型在进入下一个周期之前运行的训练步数）。由于 `Dataset` 会生成批次数据，因此该代码段不需要 `batch_size`。

数据集也可用于验证：

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32).repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32).repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset,
          validation_steps=3)

Epoch 1/10
30/30 [==============================] - 0s 9ms/step - loss: 11.5523 - categorical_accuracy: 0.1583 - val_loss: 11.8241 - val_categorical_accuracy: 0.1146
Epoch 2/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5832 - categorical_accuracy: 0.1763 - val_loss: 11.7159 - val_categorical_accuracy: 0.1176
Epoch 3/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5534 - categorical_accuracy: 0.1816 - val_loss: 11.3228 - val_categorical_accuracy: 0.1176
Epoch 4/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5319 - categorical_accuracy: 0.1635 - val_loss: 11.4193 - val_categorical_accuracy: 0.1176
Epoch 5/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5494 - categorical_accuracy: 0.1795 - val_loss: 11.9241 - val_categorical_accuracy: 0.0625
Epoch 6/10
30/30 [==============================] - 0s 1ms/step - loss: 11.5235 - categorical_accuracy: 0.1976 - val_loss: 11.7083 - val_categorical_accuracy: 0.0441
Epoc

### 评估和预测
`tf.keras.Model.evaluate` 和 `tf.keras.Model.predict` 方法可以使用 **NumPy** 数据和 `tf.data.Dataset`。

要评估所提供数据的推理模式损失和指标，请运行以下代码：

In [12]:
data = np.random.random((1000,32))
labels = np.random.random((1000,10))

model.evaluate(data,labels,batch_size=32)

model.evaluate(dataset, steps=30)

30/30 [==============================] - 0s 3ms/step - loss: 11.5311 - categorical_accuracy: 0.2010


[11.5310528755188, 0.20104167]

要在所提供数据（采用 NumPy 数组形式）的推理中预测最后一层的输出，请运行以下代码：

In [13]:
result = model.predict(data,batch_size=32)
print(result.shape)

(1000, 10)


In [19]:
print(result[:10])

[[0.10712593 0.10614926 0.11914968 0.10438284 0.09777807 0.08018789
  0.08364492 0.11341213 0.09596829 0.09220092]
 [0.05993045 0.06431799 0.11444446 0.10143398 0.08408388 0.1351158
  0.0910539  0.13234802 0.10135642 0.11591516]
 [0.09521271 0.11232687 0.09991261 0.09509832 0.10266942 0.09157606
  0.10383549 0.0972802  0.10034923 0.10173913]
 [0.10156393 0.09303255 0.10199981 0.09310558 0.09298146 0.08690888
  0.10563116 0.1067415  0.1123837  0.10565142]
 [0.09313243 0.09825429 0.11126506 0.09561428 0.10602416 0.08508237
  0.11673948 0.12671322 0.07201951 0.09515522]
 [0.09622911 0.10811144 0.1045793  0.10417874 0.10168935 0.08779634
  0.10045061 0.0986843  0.09662527 0.10165551]
 [0.09230995 0.10833034 0.09842029 0.09985278 0.10446718 0.08733904
  0.1088278  0.10059937 0.08843445 0.11141869]
 [0.08911548 0.09282499 0.10479552 0.08142848 0.05217983 0.10513885
  0.11305489 0.11727671 0.10720601 0.13697924]
 [0.07613072 0.12121036 0.08221168 0.07120643 0.10866243 0.11784232
  0.11064467 

## 构建高级模型
### 函数式 API
`tf.keras.Sequential` 模型是层的简单堆叠，无法表示任意模型。使用 `Keras 函数式 API` 可以构建复杂的模型拓扑，例如：

- 多输入模型，
- 多输出模型，
- 具有共享层的模型（同一层被调用多次），
- 具有非序列数据流的模型（例如，剩余连接）。

使用函数式 API 构建的模型具有以下特征：

1. 层实例可调用并返回张量。
2. 输入张量和输出张量用于定义 `tf.keras.Model `实例。
3. 此模型的训练方式和` Sequential `模型一样。
以下示例使用函数式 API 构建一个简单的全连接网络：

In [20]:
inputs = tf.keras.Input(shape=(32,)) # Returns a placeholder tensor

#A layer instance is callable on a tensor , and returns a tensor.
x = layers.Dense(64,activation='relu')(inputs)
x = layers.Dense(64,activation='relu')(x)
predictions = layers.Dense(10,activation='softmax')(x)

在给定输入和输出的情况下实例化模型。

In [21]:
model = tf.keras.Model(inputs=inputs,outputs=predictions)

# The compile step specifies the training configuration.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

#Trains for 5 epochs
model.fit(data,labels,batch_size=32,epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 196us/sample - loss: 11.7163 - acc: 0.0920
Epoch 2/5
1000/1000 [==============================] - 0s 46us/sample - loss: 11.6079 - acc: 0.0850
Epoch 3/5
1000/1000 [==============================] - 0s 41us/sample - loss: 11.5692 - acc: 0.0930
Epoch 4/5
1000/1000 [==============================] - 0s 40us/sample - loss: 11.5568 - acc: 0.0960
Epoch 5/5
1000/1000 [==============================] - 0s 40us/sample - loss: 11.5491 - acc: 0.0930


### 模型子类化
通过对 `tf.keras.Model` 进行子类化并定义您自己的前向传播来构建完全可自定义的模型。在` __init__` 方法中创建层并将它们设置为类实例的属性。在 `call `方法中定义前向传播。

在启用` Eager Execution` 时，模型子类化特别有用，因为可以命令式地编写前向传播。

>要点：针对作业使用正确的 API。虽然模型子类化较为灵活，但代价是复杂性更高且用户出错率更高。如果可能，请首选函数式 API。

以下示例展示了使用自定义前向传播进行子类化的` tf.keras.Model`：

In [22]:
class MyModel(tf.keras.Model):
    def __init__(self,num_classes=10):
        super(MyModel,self).__init__(name='my_model')
        self.num_classes = num_classes
        
        #Define your layers here
        self.dense_1 = layers.Dense(32,activation='relu')
        self.dense_2 = layers.Dense(num_classes,activation='sigmoid')
        
    def call(self,inputs):
        #Define your forward pass here,
        #using layers you previously defined (in `__init__`).
        x = self.dense_1(inputs)
        return self.dense_2(x)
    
    def compute_output_shape(self,input_shape):
        #You need to override this fuction if you want to use the subclassed model
        # as part of a functional-style model.
        # Otherwise,this method is optional
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)
        

实例化新模型类：

In [23]:
model = MyModel(num_classes=10)

#The compile step specifies the training configuration
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

#Trains for 5 epochs
model.fit(data,labels,batch_size=32,epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 173us/sample - loss: 11.7401 - acc: 0.1040
Epoch 2/5
1000/1000 [==============================] - 0s 41us/sample - loss: 11.6712 - acc: 0.1040
Epoch 3/5
1000/1000 [==============================] - 0s 31us/sample - loss: 11.5933 - acc: 0.1070
Epoch 4/5
1000/1000 [==============================] - 0s 38us/sample - loss: 11.5640 - acc: 0.1190
Epoch 5/5
1000/1000 [==============================] - 0s 32us/sample - loss: 11.5568 - acc: 0.1110


### 自定义层
通过对 `tf.keras.layers.Layer `进行子类化并实现以下方法来创建自定义层：

- `build`：创建层的权重。使用 `add_weight` 方法添加权重。
- `call`：定义前向传播。
- `compute_output_shape`：指定在给定输入形状的情况下如何计算层的输出形状。
- 或者，可以通过实现 `get_config` 方法和 `from_config` 类方法序列化层。
下面是一个使用核矩阵实现输入 `matmul` 的自定义层示例：

In [28]:
class MyLayer(layers.Layer):

  def __init__(self, output_dim, **kwargs):
    self.output_dim = output_dim
    super(MyLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    shape = tf.TensorShape((input_shape[1], self.output_dim))
    # Create a trainable weight variable for this layer.
    self.kernel = self.add_weight(name='kernel',
                                  shape=shape,
                                  initializer='uniform',
                                  trainable=True)
    # Be sure to call this at the end
    super(MyLayer, self).build(input_shape)

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

  def compute_output_shape(self, input_shape):
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.output_dim
    return tf.TensorShape(shape)

  def get_config(self):
    base_config = super(MyLayer, self).get_config()
    base_config['output_dim'] = self.output_dim
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)

使用自定义层创建模型：

In [29]:
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')])

# The compile step specifies the training configuration
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs.
model.fit(data, labels, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 172us/sample - loss: 11.5722 - acc: 0.0950
Epoch 2/5
1000/1000 [==============================] - 0s 34us/sample - loss: 11.5561 - acc: 0.0940
Epoch 3/5
1000/1000 [==============================] - 0s 38us/sample - loss: 11.5506 - acc: 0.0990
Epoch 4/5
1000/1000 [==============================] - 0s 35us/sample - loss: 11.5487 - acc: 0.0950
Epoch 5/5
1000/1000 [==============================] - 0s 34us/sample - loss: 11.5472 - acc: 0.1020


## 回调
回调是传递给模型的对象，用于在训练期间自定义该模型并扩展其行为。您可以编写自定义回调，也可以使用包含以下方法的内置 `tf.keras.callbacks`：

- `tf.keras.callbacks.ModelCheckpoint`：定期保存模型的检查点。
- `tf.keras.callbacks.LearningRateScheduler`：动态更改学习速率。
- `tf.keras.callbacks.EarlyStopping`：在验证效果不再改进时中断训练。
- `tf.keras.callbacks.TensorBoard`：使用 `TensorBoard` 监控模型的行为。

要使用 `tf.keras.callbacks.Callback`，请将其传递给模型的 `fit` 方法：

In [30]:
callbacks = [
    #Interrupt training if 'val_loss' stop improving for over 2 epochs
    tf.keras.callbacks.EarlyStopping(patience=2,monitor='val_loss'),
    #Write TensorBoard logs to './logs' directory
    tf.keras.callbacks.TensorBoard(log_dir='./TensorBoardLog/')
]

model.fit(data,labels,batch_size=32,epochs=5,callbacks=callbacks,
         validation_data=(val_data,val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 0s 112us/sample - loss: 11.5449 - acc: 0.1070 - val_loss: 11.7121 - val_acc: 0.1400
Epoch 2/5
1000/1000 [==============================] - 0s 42us/sample - loss: 11.5436 - acc: 0.0930 - val_loss: 11.7148 - val_acc: 0.1100
Epoch 3/5
1000/1000 [==============================] - 0s 41us/sample - loss: 11.5422 - acc: 0.1110 - val_loss: 11.7158 - val_acc: 0.1400


#### 保存和恢复
##### 仅限权重

使用 `tf.keras.Model.save_weights` 保存并加载模型的权重：

In [31]:
model = tf.keras.Sequential([
    layers.Dense(64,activation='relu'),
    layers.Dense(10,activation='softmax')
])

model.compile(optimizer=tf.train.AdadeltaOptimizer(0.001),
             loss='categorical_crossentopy',
             metrics=['accuracy'])

In [32]:
#Save weights to a TensorFlow Checkpoint file
model.save_weights('./model/weights/my_model')

#Restore the model's state,
#this requires a model with the same architecture
model.load_weights('./model/weights/my_model')

Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.


In [33]:
json_string = model.to_json()
json_string

'{"class_name": "Sequential", "config": {"name": "sequential_6", "layers": [{"class_name": "Dense", "config": {"name": "dense_20", "trainable": true, "dtype": null, "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_21", "trainable": true, "dtype": null, "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}]}, "ke

In [34]:
import json
import pprint
pprint.pprint(json.loads(json_string))

{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'layers': [{'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {'dtype': 'float32'}},
                                   'bias_regularizer': None,
                                   'dtype': None,
                                   'kernel_constraint': None,
                                   'kernel_initializer': {'class_name': 'GlorotUniform',
                                                          'config': {'dtype': 'float32',
                                                                     'seed': None}},
                                   'kernel_regularizer': None,
                                   'name': 'dense